<a href="https://colab.research.google.com/github/urmidedhia/Synapse-Learning-Period/blob/NLP-week-3/Tasksentiment_rating_rnn_Urmi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color='#6629b2'>Predicting sentiment ratings with RNN using Keras</font>


## <font color='#6629b2'>Study Material</font>
- https://web.stanford.edu/~jurafsky/slp3/9.pdf
- https://www.youtube.com/watch?v=UNmqTiOnRfg
- https://www.youtube.com/watch?v=WCUNPb-5EYI
- https://www.youtube.com/watch?v=OuYtk9Ymut4

## <font color='#6629b2'>Dataset</font>

The Large Movie Review Datasetconsists of 50,000 movie reviews from [IMDB](http://www.imdb.com/). The ratings are on a 1-10 scale, but the dataset specifically contains "polarized" reviews: positive reviews with a rating of 7 or higher, and negative reviews with a rating of 4 or lower. There are an equal number of positive and negative reviews. 

You can download the dataset from: https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [40]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [41]:
import pandas as pd
import numpy as np

In [42]:
'''Load the dataset into variable "reviews". You can truncate the dataset to keep a few hundred records if it's
    taking too long to process/train. Keep in mind, bigger the dataset, higher the accuracy score!
'''

reviews = pd.read_csv('/content/gdrive/MyDrive/NLP_Week3.zip (Unzipped Files) (1)/week3/IMDB Dataset.csv/IMDB Dataset.csv', nrows = 1000)


reviews

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
995,Nothing is sacred. Just ask Ernie Fosselius. T...,positive
996,I hated it. I hate self-aware pretentious inan...,negative
997,I usually try to be professional and construct...,negative
998,If you like me is going to see this in a film ...,negative


In [43]:
#Split the data into train and split
X = reviews[['review']]
y = reviews[['sentiment']]
from sklearn.model_selection import train_test_split
train_reviews, test_reviews, train_sentiment, test_sentiment = train_test_split(X, y, test_size = 0.3, random_state = 20)

In [44]:
# convert the above train test datasets into a pandas dataframe. You shoud have 4 dataframes.
# Name them train_reviews, test_reviews, train_sentiment, test_sentiment.
# In train_sentiment and test_sentiment, convert "positive" to 1 and "negative" to 0
for df in [train_sentiment, test_sentiment]:
  df['sentiment'] = df['sentiment'].replace(['positive'],1)
  df['sentiment'] = df['sentiment'].replace(['negative'],0)

test_sentiment

,sentiment
890,1
694,1
798,0
147,1
858,1
...,...
257,0
591,0
377,0
938,0


In [45]:
train_reviews.head()

,review
140,"Before I begin, let me get something off my ch..."
409,A great Bugs Bunny cartoon from the earlier ye...
63,"Besides being boring, the scenes were oppressi..."
178,An unforgettable masterpiece from the creator ...
732,"They filmed this movie out on long Island, whe..."


## <font color='#6629b2'>Preparing the data</font>

###  <font color='#6629b2'>Tokenization</font>

The first preprocessing step is to tokenize each of the reviews into (lowercased) individual words, since the models will encode the reviews at the word level (rather than subword units like characters, for example). For this we'll use [spaCy](https://spacy.io/), which is a fast and extremely user-friendly library that performs various language processing tasks. Once you load a spaCy model for a particular language, you can provide any text as input to the model, e.g. encoder(text) and access its linguistic features.

In [46]:
'''Lowercase and tokenise all the reviews in train_reviews using spacy'''

import spacy
encoder = spacy.load('en_core_web_sm')

def text_to_tokens(text_seqs):
    tokenized = [[word.lower_ for word in encoder(text)] for text in text_seqs]
    return tokenized
    #complete this function that lowers and tokenizes the reviews


train_reviews['Tokenized_Review'] = text_to_tokens(train_reviews['review'])
train_reviews[['review','Tokenized_Review']][:10]

,review,Tokenized_Review
140,"Before I begin, let me get something off my ch...","[before, i, begin, ,, let, me, get, something,..."
409,A great Bugs Bunny cartoon from the earlier ye...,"[a, great, bugs, bunny, cartoon, from, the, ea..."
63,"Besides being boring, the scenes were oppressi...","[besides, being, boring, ,, the, scenes, were,..."
178,An unforgettable masterpiece from the creator ...,"[an, unforgettable, masterpiece, from, the, cr..."
732,"They filmed this movie out on long Island, whe...","[they, filmed, this, movie, out, on, long, isl..."
187,While I count myself as a fan of the Babylon 5...,"[while, i, count, myself, as, a, fan, of, the,..."
30,"Taut and organically gripping, Edward Dmytryk'...","[taut, and, organically, gripping, ,, edward, ..."
333,"Often laugh out loud funny play on sex, family...","[often, laugh, out, loud, funny, play, on, sex..."
881,"Herculis Puaro is, in general, a well establis...","[herculis, puaro, is, ,, in, general, ,, a, we..."
414,"I feel dumber after watching the first 20 min,...","[i, feel, dumber, after, watching, the, first,..."


###  <font color='#6629b2'>Lexicon</font>

Then we need to assemble a lexicon (aka vocabulary) of words that the model needs to know. Each tokenized word in the reviews is added to the lexicon, and then each word is mapped to a numerical index that can be read by the model. Since large datasets may contain a huge number of unique words, it's common to filter all words occurring less than a certain number of times, and replace them with some generic &lt;UNK&gt; token. The min_freq parameter in the function below defines this threshold. When assigning the indices, the number 1 will represent unknown words. The number 0 will represent "empty" word slots, which is explained below. Therefore "real" words will have indices of 2 or higher.

In [47]:
'''Count tokens (words) in texts and add them to the lexicon'''

import pickle

def make_lexicon(token_seqs, min_freq=1, use_padding=False):
    # First, count how often each word appears in the text. Save this count in a dictionary called token_counts
    token_counts = {}
    for text in token_seqs:
      for token in text:
        if token in token_counts:
          token_counts[token] += 1
        else:
          token_counts[token] = 1
  
    # Then, assign each word to a numerical index, i.e save  all these words in a list. Filter words that occur less than or equal to min_freq times.
    
    lexicon = [token for token, _ in token_counts.items() if token_counts[token] >= min_freq]
    
    #create a dictionary lexicon that maps each word to its index. Note that indexes will start from 2,  index 0 is saved for padding and index 1 for unknown words ('<UNK>')
    lexicon = {token:index+2 for index, token in enumerate(lexicon)}
    
    lexicon[u'<UNK>'] = 1 # Unknown words are those that occur fewer than min_freq times
    lexicon_size = len(lexicon)
    print(lexicon)

    print("LEXICON SAMPLE ({} total items):".format(lexicon_size))
    print(dict(list(lexicon.items())[:20]))
    
    return lexicon

lexicon = make_lexicon(token_seqs=train_reviews['Tokenized_Review'], min_freq=1)

{'before': 2, 'i': 3, 'begin': 4, ',': 5, 'let': 6, 'me': 7, 'get': 8, 'something': 9, 'off': 10, 'my': 11, 'chest': 12, ':': 13, "'m": 14, 'a': 15, 'huge': 16, 'fan': 17, 'of': 18, 'john': 19, 'eyres': 20, "'": 21, 'first': 22, 'film': 23, 'project': 24, 'shadowchaser': 25, '.': 26, 'the': 27, 'b': 28, '-': 29, 'grade': 30, 'cross': 31, 'both': 32, 'terminator': 33, '&': 34, 'die': 35, 'hard': 36, 'may': 37, 'not': 38, 'be': 39, 'work': 40, 'cinematic': 41, 'genius': 42, 'but': 43, 'is': 44, 'hugely': 45, 'entertaining': 46, 'action': 47, 'that': 48, 'became': 49, 'cult': 50, 'hit': 51, '(': 52, 'spawned': 53, 'two': 54, 'sequels': 55, 'spin': 56, 'off).<br': 57, '/><br': 58, '/>judge': 59, 'and': 60, 'jury': 61, 'begins': 62, 'with': 63, 'joseph': 64, 'meeker': 65, 'convicted': 66, 'killer': 67, 'who': 68, 'was': 69, 'sent': 70, 'to': 71, 'death': 72, 'row': 73, 'following': 74, 'his': 75, 'capture': 76, 'after': 77, 'so': 78, 'called': 79, '"': 80, 'bloody': 81, 'shootout': 82, 'whi

###  <font color='#6629b2'>From strings to numbers</font>

Once the lexicon is built, we can use it to transform each review from a list of string tokens into a list of numerical indices.

In [48]:
'''Convert each review from a list of tokens to a list of numbers (indices)'''

def tokens_to_idxs(token_seqs, lexicon): 
    #complete this function to return a list of indexed tokens 
    idx_seqs = [[lexicon[token] if token in lexicon else lexicon['<UNK>'] for token in text] for text in token_seqs]
    return idx_seqs

train_reviews['Review_Idxs'] = tokens_to_idxs(token_seqs=train_reviews['Tokenized_Review'], lexicon=lexicon)
                                   
train_reviews[['Tokenized_Review', 'Review_Idxs']][:10]

,Tokenized_Review,Review_Idxs
140,"[before, i, begin, ,, let, me, get, something,...","[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 3, 14..."
409,"[a, great, bugs, bunny, cartoon, from, the, ea...","[15, 252, 253, 254, 255, 182, 27, 256, 257, 22..."
63,"[besides, being, boring, ,, the, scenes, were,...","[325, 101, 326, 5, 27, 221, 327, 328, 60, 329,..."
178,"[an, unforgettable, masterpiece, from, the, cr...","[106, 355, 356, 182, 27, 357, 18, 27, 358, 18,..."
732,"[they, filmed, this, movie, out, on, long, isl...","[438, 439, 288, 330, 166, 220, 440, 441, 5, 10..."
187,"[while, i, count, myself, as, a, fan, of, the,...","[95, 3, 493, 494, 147, 15, 17, 18, 27, 495, 49..."
30,"[taut, and, organically, gripping, ,, edward, ...","[530, 60, 531, 532, 5, 533, 534, 142, 535, 44,..."
333,"[often, laugh, out, loud, funny, play, on, sex...","[734, 735, 166, 736, 307, 737, 220, 114, 5, 42..."
881,"[herculis, puaro, is, ,, in, general, ,, a, we...","[797, 798, 44, 5, 215, 707, 5, 15, 239, 799, 2..."
414,"[i, feel, dumber, after, watching, the, first,...","[3, 841, 842, 77, 133, 27, 22, 843, 844, 5, 84..."


In [49]:
'''Encode reviews (train_reviews['Review_Idxs']) as bag-of-words vectors'''

import numpy 

def idx_seqs_to_bows(idx_seqs, matrix_length):
    #complete the function to return an array having bag-of-words vectors of the encoded reviews
    # hint: numpy.bincount()
    bow_seqs = numpy.array([numpy.bincount(numpy.array(seq), minlength = matrix_length) 
                            for seq in idx_seqs])
    return bow_seqs
    

bow_train_reviews = idx_seqs_to_bows(train_reviews['Review_Idxs'], 
                                     matrix_length=len(lexicon) + 1) #add one to length for padding)

print("TRAIN INPUT:\n", bow_train_reviews)
print("SHAPE:", bow_train_reviews.shape, "\n")

#Showing an example mapping string words to counts
lexicon_lookup = {idx: lexicon_item for lexicon_item, idx in lexicon.items()}
lexicon_lookup[0] = ""
pd.DataFrame([(lexicon_lookup[idx], count) for idx, count in enumerate(bow_train_reviews[0])], 
                 columns=['Word', 'Count'])

TRAIN INPUT:
 [[0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 2 1 1]]
SHAPE: (700, 16476) 



,Word,Count
0,,0
1,<UNK>,0
2,before,1
3,i,2
4,begin,1
...,...,...
16471,scarecrow,0
16472,slayer,0
16473,toons,0
16474,pumpkin,0


##  <font color='#6629b2'>Building a Recurrent Neural Network </font>



###  <font color='#6629b2'>Numerical lists to matrices</font>

The input representation for the RNN explicitly encodes the order of words in the review. We'll return to the lists of the word indices contained in train_reviews['Review_Idxs']. The input to the model will be these number sequences themselves. We need to put all the reviews in the training set into a single matrix, where each row is a review and each column is a word index in that sequence. This enables the model to process multiple sequences in parallel (batches) as opposed to one at a time. Using batches significantly speeds up training. However, each review has a different number of words, so we create a padded matrix equal to the length on the longest review in the training set. For all reviews with fewer words, we prepend the row with zeros representing an empty word position. This is why the number 0 was not assigned as a word index in the lexicon. We can tell Keras to ignore these zeros during training.

In [50]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def pad_idx_seqs(idx_seqs):
    
    #find the biggest review's length and save it in the variable below
    max_seq_len = max([len(seq) for seq in idx_seqs])
    
    #pad all these indexed reviews and return these padded sequences
    #HINT: use pad_sequences function by keras
    padded_seq = pad_sequences(sequences = idx_seqs, maxlen = max_seq_len)
    return padded_seq

train_padded_idxs = pad_idx_seqs(train_reviews['Review_Idxs'])

print("TRAIN INPUT:\n", train_padded_idxs)
print("SHAPE:", train_padded_idxs.shape, "\n")

TRAIN INPUT:
 [[    0     0     0 ...   250   251    26]
 [    0     0     0 ...    30    13   324]
 [    0     0     0 ...   170   354    26]
 ...
 [    0     0     0 ...   252  1972   769]
 [    0     0     0 ... 15143   166    26]
 [    0     0     0 ...   330   465    26]]
SHAPE: (700, 1446) 



###  <font color='#6629b2'>Model Layers</font>
The RNN will have four layers:

**1. Input**: The input layer takes in the matrix of word indices.

**2. Embedding**: A [layer](https://keras.io/layers/embeddings/) that converts integer word indices into distributed vector representations (embeddings), which were introduced above. The difference here is that rather than plugging in embeddings from a pretrained model as before, the word embeddings will be learned inside the model itself. Thus, the input to the model will be the word indices rather than their embeddings, and the embedding values will change as the model is trained. The mask_zero=True parameter in this layer indicates that values of 0 in the matrix (the padding) will be ignored by the model.

**3. GRU**: A [recurrent (GRU) hidden layer](https://keras.io/layers/recurrent/), the central component of the model. As it observes each word in the review, it integrates the word embedding representation with what it's observed so far to compute a representation (hidden state) of the review at that timepoint. There are a few architectures for this layer - we use the GRU variation, Keras also provides LSTM or just the simple vanilla recurrent layer (see the materials at the bottom for an explanation of the difference). This layer outputs the last hidden state of the sequence (i.e. the hidden representation of the review after its last word is observed).

**4. Dense**: An output [layer](https://keras.io/layers/core/#dense) that predicts the rating for the review based on its GRU representation given by the previous layer. It has one dimension that contains a continuous value (the rating). Add a proper activation function.

###  <font color='#6629b2'>Parameters</font>

Our function for creating the RNN takes the following parameters:

**n_input_nodes**: As with the standard bag-of-words MLP, this is the number of unique words in the lexicon, plus one to account for the padding represented by 0 values. This indicates the number of rows in the embedding layer, where each row corresponds to a word.

**n_embedding_nodes**: the number of dimensions (units) in the embedding layer, which can be freely defined. Here, it is set to 300.

**n_hidden_nodes**: the number of dimensions in the GRU hidden layer. Like the embedding layer, this can be freely chosen. Here, it is set to 500.

In [51]:
'''Create the model'''

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.layers import InputLayer

def create_rnn_model(n_input_nodes, n_embedding_nodes, n_hidden_nodes):
    
    #complete this function to create a model and compile it having the 4 layers listed above.
    #Note: Layer 1 -  Technically the shape of this layer is (batch_size, len(train_padded_idxs)).
    # However, both the batch size and the length of the input matrix can be inferred from the input at training time. 
    # The batch size is implicitly included in the shape of the input, so it does not need to 
    # be specified as a dimension of the input. None can be given as placeholder for the input matrix length.
    # By defining it as None, the model is flexible in accepting inputs with different lengths.
    input_layer = Input(shape=(None,))
    
    # Layer 2
    embedding_layer = Embedding(input_dim=n_input_nodes, output_dim=n_embedding_nodes, mask_zero=True)(input_layer)

    # Layer 3
    gru_layer = GRU(units = n_hidden_nodes)(embedding_layer)

    # Layer 4
    output_layer = Dense(units = 1)(gru_layer)

    #Specify which layers are input and output, compile model with loss and optimization functions
    model = Model(inputs=[input_layer], outputs=output_layer)
    model.compile(loss="mean_squared_error", optimizer='adam')
    return model


In [52]:
rnn_model = create_rnn_model(n_input_nodes=len(lexicon) + 1, n_embedding_nodes=300, n_hidden_nodes=500)

###  <font color='#6629b2'>Training</font>

In [53]:
'''
Train the rnn_model using the padded sequences and y=train_reviews['Rating'].
You need  to convert train_reviews['Rating'] to tensor before passing it as an argument
Hint: tf.convert_to_tensor
batch_size=20, epochs=5
'''
train_senti_tensor = tf.convert_to_tensor(train_sentiment.values, dtype = np.int64)
rnn_model.fit(x=train_padded_idxs, y=train_senti_tensor, batch_size=20, epochs=5)

Epoch 1/5
35/35 [==============================] - 444s 13s/step - loss: 0.2856
Epoch 2/5
35/35 [==============================] - 452s 13s/step - loss: 0.1728
Epoch 3/5
35/35 [==============================] - 438s 13s/step - loss: 0.0828
Epoch 4/5
35/35 [==============================] - 439s 13s/step - loss: 0.0446
Epoch 5/5
35/35 [==============================] - 429s 12s/step - loss: 0.0229


###  <font color='#6629b2'>Prediction</font>

In [54]:
'''Put test reviews in padded matrix just how we did for train_reviews'''
test_reviews['Tokenized_Review'] = text_to_tokens(test_reviews['review'])
# lexicon_test = make_lexicon(token_seqs=test_reviews['Tokenized_Review'], min_freq=1)
test_reviews['Review_Idxs'] = tokens_to_idxs(token_seqs=test_reviews['Tokenized_Review'], lexicon=lexicon)
test_padded_idxs = pad_idx_seqs(test_reviews['Review_Idxs'])

print("TEST INPUT:\n", test_padded_idxs)
print("SHAPE:", test_padded_idxs.shape, "\n")

TEST INPUT:
 [[   0    0    0 ...  278 1396   26]
 [   0    0    0 ...   26   38    1]
 [   0    0    0 ...   26 5926   26]
 ...
 [   0    0    0 ...   30   13 4543]
 [   0    0    0 ...    1 1470  343]
 [   0    0    0 ...  727    1   26]]
SHAPE: (300, 1179) 



In [55]:
'''Predict the ratings '''

#Since ratings are integers, need to round predicted rating to nearest integer
test_reviews['RNN_Pred_Rating'] = numpy.round(rnn_model.predict(test_padded_idxs)[:,0]).astype(int)

In [56]:
test_reviews
import sklearn
score = sklearn.metrics.accuracy_score(y_true = test_sentiment['sentiment'], y_pred = test_reviews['RNN_Pred_Rating'])
score

0.6266666666666667

###  <font color='#6629b2'>Evaluation</font>

In [57]:
'''Evaluate the model with R^2'''

# print the r2 score
from sklearn.metrics import r2_score 
final_score = r2_score(y_true = test_sentiment['sentiment'], y_pred = test_reviews['RNN_Pred_Rating'])
print(final_score)

-0.49659257939512713


On the full test dataset of 25,000 reviews, the $R^2$ for this model is 0.622525. So the RNN outperforms the continuous bag-of-words MLP as well as the standard bag-of-words approach.
Your score might not be good because we're training on only 100-200 reviews.

### <font color='#6629b2'>Visualizing data inside the model</font>

To help visualize the data representation inside the model, we can look at the output of each layer in a model individually. Keras' Functional API lets you derive a new model with the layers from an existing model, so you can define the output to be a layer below the output layer in the original model. Calling predict() on this new model will produce the output of that layer for a given input. Of course, glancing at the numbers by themselves doesn't provide any interpretation of what the model has learned (although there are opportunities to [interpret these values](https://medium.com/civis-analytics/interpreting-and-visualizing-neural-networks-for-text-processing-e9dff0da9c22), but seeing them verifies the model is just a series of transformations from one matrix to another. The model stores its layers as the list model.layers, and you can retrieve specific layer by its position index in the model.

In [58]:
'''Showing the output of the RNN embedding layer (second layer) for the test reviews'''

embedding_layer = Model(inputs=rnn_model.layers[0].input, 
                        outputs=rnn_model.layers[1].output) #embedding layer is 2nd layer (index 1)
embedding_output = embedding_layer.predict(test_padded_idxs)
print("EMBEDDING LAYER OUTPUT SHAPE:", embedding_output.shape)
print(embedding_output[0])

EMBEDDING LAYER OUTPUT SHAPE: (300, 1179, 300)
[[ 0.04452343  0.03647571  0.03306024 ... -0.04267442  0.00410755
  -0.02598894]
 [ 0.04452343  0.03647571  0.03306024 ... -0.04267442  0.00410755
  -0.02598894]
 [ 0.04452343  0.03647571  0.03306024 ... -0.04267442  0.00410755
  -0.02598894]
 ...
 [ 0.01674429  0.01562578  0.04162631 ...  0.04001122  0.02232656
   0.02147009]
 [-0.01142752  0.027111    0.02257015 ... -0.03155704  0.01093339
   0.0128407 ]
 [ 0.00016451 -0.00806309 -0.01970505 ...  0.02106707  0.02488177
   0.00260685]]


## <font color='#6629b2'>Conclusion</font>

As mentioned above, the models shown here could be applied to any task where the goal is to predict a score for a particular sequence. For ratings prediction, this output is ordinal, but it could also be categorical with a few simple changes to the output layer of the model. 

## <font color='#6629b2'>More resources</font>

Yoav Goldberg's book [Neural Network Methods for Natural Language Processing](http://www.morganclaypool.com/doi/abs/10.2200/S00762ED1V01Y201703HLT037) is a thorough introduction to neural networks for NLP tasks in general.

If you'd like to learn more about what Keras is doing under the hood, there is a [Theano tutorial](http://deeplearning.net/tutorial/lstm.html) that also applies an RNN to sentiment prediction, using the same dataset here

Andrej Karpathy's blog post [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) is very helpful for understanding the mathematical details of an RNN, applied to the task of language modeling. It also provides raw Python code with an implementation of the backpropagation algorithm.

TensorFlow also has an RNN language model [tutorial](https://www.tensorflow.org/versions/r0.12/tutorials/recurrent/index.html) using the Penn Treebank dataset

Chris Olah provides a good [explanation](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) of how LSTM RNNs work (this explanation also applies to the GRU model used here)

Denny Britz's [tutorial](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) documents well both the technical details of RNNs and their implementation in Python.